## Question 1

#### <ins>Equation A</ins>
The equation $x+y-z=0$ is valid in a linear program because it is a linear equation.

#### <ins>Equation B</ins>
The equation $x \le \frac{100}{y}$ is not valid in a linear program because it is a non-linear function.


#### <ins>Equation C</ins>
The equation $3x+2y \le \sqrt{5}$ is valid in a linear program because it is a linear function.


#### <ins>Equation D</ins>
The equation $\sqrt{5}x+2y = 50$ is valid in a linear program because it is a linear function.


#### <ins>Equation E</ins>
The equation $\sqrt{5x}+10y = 100$ is not valid in a linear program because it is a non-linear function as a result of the $\sqrt{5x}$.


#### <ins>Equation F</ins>
The equation $x^{2}+y^{2} \ge 45$ is not valid in a linear program because it contains quadratic terms.


## Question 6